# DEA Notebook 

![alt text](../dea-gallery-crop.png "DEA")

### [DEA](https://dea.destine.eu/web) is a Content creation service and no-code platform for DestinE storytelling and data visualization

## Generate a Cloud Optimized GeoTiff from NetCDF

This notebook can be used to generate a Cloud Optimized GeoTIFF (COG) file compliant with the DEA service.

In the following example, the COG file is generated from a NetCDF file retrieved by the DestinE CacheB service (2 metre temperature)

# Install pre-requirements

In [1]:
!pip install matplotlib
!pip install numpy
!pip install xarray
!pip install rasterio
!pip install rio-cogeo
!pip install rioxarray

  Using cached rio_cogeo-5.3.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached morecantile-5.4.2-py3-none-any.whl.metadata (7.0 kB)
Using cached rio_cogeo-5.3.6-py3-none-any.whl (20 kB)
Using cached morecantile-5.4.2-py3-none-any.whl (49 kB)


# Download the sample NetCDF file and save it in the input folder

DESP CacheB will be used to download an [ERA5](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5)  t2m (2 metre temperature) data selecting 2024-08-01 as date. 

The data will be saved locally in NetCDF format.

>**_NOTE_**: the temporary password is valid for a limited period of time and needs to be regenerated and reconfigured periodically by running the cells above.

The following file will be downlaoded locally: **t2m_era5_20240801.nc**

In [2]:
%%capture cap
%run ./../../cacheb/cacheb-authentication.py

Username:  dea@alia-space.com
Password:  ········


## You can use your own data!

>**_NOTE_:** You can use your own NetCDF file, but please consider that the NetCDF file must have latitude and longitude coordinates defined in the range -180,180,-90,90. 

In case the original file has a different definition, applying, for instance, the following CDO command:
 
 ```
cdo sellonlatbox,-180,180,-90,90 <input_file.nc> <output_file.nc>
```

More info on CDO (Climate Data Operators) are available [here](https://code.mpimet.mpg.de/projects/cdo)

In [3]:
output_1 = cap.stdout.split('}\n')
token = output_1[-1][0:-1]

from pathlib import Path
with open(Path.home() / ".netrc", "a") as fp:
    print
    fp.write(token)

## Define variables

In [6]:
file_path = "t2m_era5_20240801.nc"
tif_path = "t2m_era5_20240801.tiff"

## Get data

From Destine Platform ChacheB service

In [8]:
import xarray as xr


data = xr.open_dataset(
        "https://cacheb.dcms.destine.eu/era5/reanalysis-era5-single-levels-v0.zarr",
        engine="zarr",
        storage_options={"client_kwargs": {"trust_env": "true"}},
        chunks={},
    )

# Convert to Celsius

t2m = data.t2m.astype("float32") - 273.15
t2m.attrs["units"] = "C"

t2m = t2m.assign_coords(longitude=(t2m['longitude'] % 360))  # First ensure longitude is in 0-360 range
t2m['longitude'] = xr.where(t2m['longitude'] > 180, t2m['longitude'] - 360, t2m['longitude'])  # Then shift to -180 to 180

# Sort by longitude to maintain the proper order
t2m = t2m.sortby('longitude')

# t2m['longitude'] = xr.where(t2m['longitude'] > 180, t2m['longitude'] - 360, t2m['longitude'])



# get a single time step
t2m_slice = t2m.sel(valid_time="2024-08-01T12:00:00")

print(f"Data to download...")
print(t2m_slice)

t2m_slice = t2m_slice.compute()
# save as NetCDF
t2m_slice.to_netcdf(file_path)

print(f"Your file {file_path} is ready!")

Data to download...
<xarray.DataArray 't2m' (latitude: 721, longitude: 1440)> Size: 4MB
dask.array<getitem, shape=(721, 1440), dtype=float32, chunksize=(64, 64), chunktype=numpy.ndarray>
Coordinates:
    entireAtmosphere  float32 4B 0.0
  * latitude          (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
    number            int64 8B 0
    surface           float64 8B 0.0
    valid_time        datetime64[ns] 8B 2024-08-01T12:00:00
  * longitude         (longitude) float64 12kB -179.8 -179.5 ... 179.8 180.0
Attributes:
    units:    C
Your file t2m_era5_20240801.nc is ready!


## Wait until your NetCDF file appear in your workspace before executing the next cell. 

>**_NOTE:_** The download of the file should take about 1 minute. The file _t2m_era5_20240801.nc_ should appear in the notebook workspace

# Convert NetCDF to GeoTiff

The following file will be downlaoded locally: **t2m_era5_20240801.tiff**

## 2. Import dependencies 

In [9]:
import xarray as xr
import rioxarray

## 3. Open dataset and select variable (e.g. var='t2m')

In [10]:
var = 't2m'
data = xr.open_dataset(file_path)
data = data[var]

## 4. Apply CRS (assuming the coordinates are longitude and latitude)

In [11]:

var = data.rename({'latitude': 'y', 'longitude': 'x'})
var.rio.write_crs("EPSG:4326", inplace=True)


<xarray.DataArray 't2m' (y: 721, x: 1440)> Size: 4MB
[1038240 values with dtype=float32]
Coordinates:
    entireAtmosphere  float32 4B ...
  * y                 (y) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
    number            int64 8B ...
    surface           float64 8B ...
    valid_time        datetime64[ns] 8B ...
  * x                 (x) float64 12kB -179.8 -179.5 -179.2 ... 179.8 180.0
    spatial_ref       int64 8B 0
Attributes:
    units:    C

## 5. Only for not global datasets

If data contains NaN, replace it with -9999 to make them transparent afterwards (as soon as you then map this value to black in the colormap text file)


In [12]:
var_filled = var.fillna(-9999)

## 6. Rotate data 

it can be necessary to flip data of 180 degrees; verify if this step is necessary to your case

```
var_filled = var_filled[::-1, ::]
```

## 7. Save the transformed data to GeoTiff (intermediate step)

Wait untill the tiff file is created (it could take a while)

In [13]:
var_filled.rio.to_raster(tif_path)

# Apply the colormap to the generated Tiff in grayscale

Matplotlib will be used to achieve this.

The following file will be downlaoded locally: **t2m_era5_20240801_rgb.tiff**

## 1. Load the colormap from the file

The file colormap.txt contains a colormap suitable for temperature in Celsius.

In [14]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Load the colormap from the file
def load_colormap(colormap_file):
    colormap = []
    values = []
    with open(colormap_file, 'r') as f:
        for line in f:
            # Skip empty lines
            if not line.strip():
                continue
            parts = line.strip().split(maxsplit=1)  # Split only at the first space
            value = float(parts[0])
            # Ensure correct parsing of the RGB values
            rgb = tuple(map(int, parts[1].replace(',', ' ').split()))
            values.append(value)
            colormap.append(rgb)
    return values, colormap

values, colormap = load_colormap('colormap.txt')

## 2. Create a custom colormap for matplotlib

In [15]:
def create_colormap(values, colormap):
    norm_values = np.linspace(0, 1, len(values))
    colors = np.array(colormap) / 255.0  # Normalize RGB to [0, 1] range
    return ListedColormap(colors), norm_values

cmap, norm_values = create_colormap(values, colormap)

## 3. Open the grayscale TIFF

In [16]:
with rasterio.open(tif_path) as src:
    grayscale = src.read(1)  # Read the first band

## 4. Apply the colormap

Interpolate the grayscale values to teh colormap

A new Tiff will be generated

In [17]:
output_rgb_tiff = "t2m_era5_20240801_rgb.tiff"

In [18]:
def apply_colormap_to_tiff(input_tiff, output_tiff, values, colormap):
    with rasterio.open(input_tiff) as src:
        grayscale = src.read(1)  # Read the first band (grayscale)
        
        # Apply the colormap to the grayscale data
        normalized = (grayscale - np.min(grayscale)) / (np.max(grayscale) - np.min(grayscale))  # Normalize
        cmap = ListedColormap(np.array(colormap) / 255.0)  # Convert colormap to matplotlib format
        rgb_image = cmap(normalized)

        # Write the RGB image to a new TIFF
        height, width, _ = rgb_image.shape
        profile = src.profile
        profile.update(count=3, dtype=rasterio.uint8)  # Update profile to handle RGB output

        with rasterio.open(output_tiff, 'w', **profile) as dst:
            dst.write((rgb_image[:, :, 0] * 255).astype(np.uint8), 1)
            dst.write((rgb_image[:, :, 1] * 255).astype(np.uint8), 2)
            dst.write((rgb_image[:, :, 2] * 255).astype(np.uint8), 3)

# Apply the colormap (assuming 'values' and 'colormap' were already loaded)
apply_colormap_to_tiff(tif_path, output_rgb_tiff, values, colormap)

# Convert TIFF to COG optimized for web

The following file will be downlaoded locally: **t2m_era5_20240801_web_optimized.tiff**

First of all, import the needed dependencies

In [19]:
import rasterio
from rasterio.enums import Resampling
from rio_cogeo.cogeo import cog_validate, cog_translate
from rio_cogeo.profiles import cog_profiles


## 1. Define paths to your files

In [20]:
output_cog = "t2m_era5_20240801_cog.tiff"
output_cog_web_optimized = "t2m_era5_20240801_web_optimized.tiff"

## 2. Convert the GeoTiff to COG

In [21]:
cog_profile = cog_profiles.get("deflate")  # You can choose other profiles such as "jpeg", "lzw"

config = {
    "blocksize": 512,  # You can customize the blocksize
    "resampling": "average"  # Pass the resampling method as a string
}

with rasterio.open(output_rgb_tiff) as src:
    # Create COG
    cog_translate(
        src,
        output_cog,
        cog_profile,
        config=config,
        nodata=src.nodata,
        overview_level=8,  # Define the overview level
        overview_resampling="average",  # Pass the resampling method as a string
    )

print("COG created successfully!")

Reading input: <open DatasetReader name='t2m_era5_20240801_rgb.tiff' mode='r'>

Adding overviews...
Updating dataset tags...
Writing output to: t2m_era5_20240801_cog.tiff


COG created successfully!


## 3. Convert COG to web-optimized COG using subprocess

In [22]:
import subprocess

subprocess.run([
    "rio", "cogeo", "create", 
    output_cog, 
    output_cog_web_optimized,
    "--blocksize", "512",
    "--overview-resampling", "average",
    "--overview-level", "8",
    "--web-optimized"
], check=True)

print("Web-optimized COG created successfully!")


Defining overview's `blocksize` to match the high resolution `blocksize`: 512


Reading input: /home/jovyan/desp-lab/dea-test/netcdf2cog/t2m_era5_20240801_cog.tiff

Adding overviews...
Updating dataset tags...
Writing output to: /home/jovyan/desp-lab/dea-test/netcdf2cog/t2m_era5_20240801_web_optimized.tiff


Web-optimized COG created successfully!


**NOTE**: The global COG file may result in missing the data over the poles, as those regions introduce distortions in the visualization.

# Upload the Web-Optimized COG on DEA!

You can now download the generated  Web-Optimized COG file _t2m_era5_20240801_web_optimized.tiff_ and upload it on [DEA Story Editor](https://dea.destine.eu/stories/editor).
The asset can be included in your stories.


![alt text](dea_upload_asset.png "Upload Assets on DEA")


![alt text](dea_cog_asset.png "COG Asset on DEA")

# Alternative methods
## Apply the colormap with GDAL

First transform to virtual VRT file and then apply colormap

```
gdal_translate -of VRT <tif_filename.tif> <vrt_filename.vrt>
gdaldem color-relief  <vrt_filename.vrt> <path_to_colormap.txt>  <output_vrt_filename.vrt>
```

## Convert the colored VRT to COG

```
gdal_translate -of COG  <output_vrt_filename.vrt> <output_cog_filename.tif>
```

## Convert the COG to WEB-OPTIMISED using rio-cogeo

```
rio cogeo create  <output_cog_filename.tif> <output_cog_optimised_filename.tif>  --blocksize 512 --overview-resampling average --overview-level 8 --web-optimized
```

This is a valid processing for Global datasets. 
In different cases, you may need to consult the rio-cogeo guidelines to optimise your custom data and avoid distortions in visualization. 

Please take a look at [rio-cogeo](https://cogeotiff.github.io/rio-cogeo/) for details.
